In [1]:
import cv2
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow_docs.vis import embed
import numpy as np


# Import matplotlib libraries
# from matplotlib import pyplot as plt
# from matplotlib.collections import LineCollection
# import matplotlib.patches as patches

# import imageio
# from IPython.display import HTML, display


In [2]:
def img_crop (image):
  height=image.shape[0]
  width=image.shape[1]
  h_lower=int(height/2+ (0.45*height))
  h_upper=int(height/2- (0.38*height))
  w_left=int(width/2-(0.3*width))
  w_right=int(width/2+(0.2*width))
  #print(h_upper)
  image = image[h_upper:h_lower, w_left:w_right, :]

  return image

In [3]:
import os
import numpy as np
import cv2
from glob import glob

def create_dir(path):
    try:
        if not os.path.exists(path):
            os.makedirs(path)
    except OSError:
        print(f"ERROR: creating directory with name {path}")
def save_frame(video_path, save_dir, gap=30):
    name = video_path.split("/")[-1].split(".")[0].lower()
    subname=name[6:]
    
  
  
    print(name)
    print(subname)
    if (name.find("_bhu") != -1) :
          save_path = os.path.join(save_dir, "Bhujangasana",subname)  
    elif (name.find("_padam") != -1) or (name.find("_padma") != -1):
          save_path = os.path.join(save_dir, "Padamasana",subname)
    elif (name.find("_shav") != -1):
          save_path = os.path.join(save_dir, "Shavasana",subname)
    elif (name.find("_trik") != -1):
          save_path = os.path.join(save_dir, "Trikonasana",subname)
    elif (name.find("_vriksh") != -1):
          save_path = os.path.join(save_dir, "Vrikshasana",subname)
    elif (name.find("_tad") != -1):
        save_path = os.path.join(save_dir, "Tadasana",subname)
    else:
        print(subname)
    print(save_path)
    
    create_dir(save_path)

    cap = cv2.VideoCapture(video_path)
    #fps = cap.get( cv2.CAP_PROP_FPS ) 
    #print(fps)
    idx = 0

    while True:
        ret, frame = cap.read()

        if ret == False:
            cap.release()
            break

        if (idx>=15 and (idx-gap//2) % gap == 0):
            image=img_crop(frame)
            cv2.imwrite(f"{save_path}/{idx}.png", image)

        idx += 1
if __name__ == "__main__":
    video_paths = glob("Yoga_Vid_Collected/train/*")
    save_dir = "dataset/train"
    

    for path in video_paths:
        save_frame(path, save_dir, gap=30)
    
    video_paths = glob("Yoga_Vid_Collected/test/*")
    save_dir = "dataset/test"
    
    for path in video_paths:
        save_frame(path, save_dir, gap=30)

train\abhay_bhuj
abhay_bhuj
dataset/train\Bhujangasana\abhay_bhuj
train\abhay_padam
abhay_padam
dataset/train\Padamasana\abhay_padam
train\abhay_shav
abhay_shav
dataset/train\Shavasana\abhay_shav
train\abhay_tada
abhay_tada
dataset/train\Tadasana\abhay_tada
train\abhay_trik
abhay_trik
dataset/train\Trikonasana\abhay_trik
train\abhay_vriksh
abhay_vriksh
dataset/train\Vrikshasana\abhay_vriksh
train\ameya_bhuj
ameya_bhuj
dataset/train\Bhujangasana\ameya_bhuj
train\ameya_padam
ameya_padam
dataset/train\Padamasana\ameya_padam
train\ameya_shav
ameya_shav
dataset/train\Shavasana\ameya_shav
train\ameya_tada
ameya_tada
dataset/train\Tadasana\ameya_tada
train\ameya_trik
ameya_trik
dataset/train\Trikonasana\ameya_trik
train\ameya_vriksh
ameya_vriksh
dataset/train\Vrikshasana\ameya_vriksh
train\bhumi_bhuj
bhumi_bhuj
dataset/train\Bhujangasana\bhumi_bhuj
train\bhumi_padam
bhumi_padam
dataset/train\Padamasana\bhumi_padam
train\bhumi_shav
bhumi_shav
dataset/train\Shavasana\bhumi_shav
train\bhumi_tada

In [7]:
# Load the input image.
def image_to_movenet(path,keypts_labels):
    
    if (path.find("Bhujangasana") != -1) :
          label="Bhujangasana"  
    elif (path.find("Padamasana") != -1):
          label="Padamasana"
    elif (path.find("Shavasana") != -1):
          label="Shavasana"
    elif (path.find("Trikonasana") != -1):
          label="Trikonasana"
    elif (path.find("Vrikshasana") != -1):
          label="Vrikshasana"
    elif (path.find("Tadasana") != -1):
          label="Tadasana"
        
    image_path = path
    image = tf.io.read_file(image_path)
    image = tf.compat.v1.image.decode_jpeg(image)
    image = tf.expand_dims(image, axis=0)
    # Resize and pad the image to keep the aspect ratio and fit the expected size.
    image = tf.cast(tf.image.resize_with_pad(image, 256, 256), dtype=tf.int32)

    # Download the model from TF Hub.
    model = hub.load("https://tfhub.dev/google/movenet/singlepose/thunder/4")
    movenet = model.signatures['serving_default']

    # Run model inference.
    outputs = movenet(image)
    # Output is a [1, 1, 17, 3] tensor.
    keypoints = outputs['output_0']
    keypoints_tup=(keypoints,label)
    keypts_labels.append(keypoints_tup)
    print(path)
    print(keypts_labels)
    
    

In [8]:
import os
import time

path_lst=[]
keypts_labels_train=[]
keypts_labels_test=[]

class_list=os.listdir('dataset/train')
for name in class_list:
    for image in os.listdir(f"dataset/train/{name}"):
        for item in os.listdir(f"dataset/train/{name}/{image}"):
             add_tup=(os.path.abspath(f"dataset/train/{name}/{image}/{item}"),name)
             path_lst.append(os.path.abspath(f"dataset/train/{name}/{image}/{item}"))



path_lst = sorted(path_lst,key=os.path.getmtime)
for path in path_lst:
    image_to_movenet(path,keypts_labels_train)

# class_list=os.listdir('dataset/test')
# for name in class_list:
#     for image in os.listdir(f"dataset/test/{name}"):
#         for item in os.listdir(f"dataset/test/{name}/{image}"):
#              add_tup=(os.path.abspath(f"dataset/test/{name}/{image}/{item}"),name)
#              path_lst.append(os.path.abspath(f"dataset/test/{name}/{image}/{item}"))



# path_lst = sorted(path_lst,key=os.path.getmtime)
# for path in path_lst:
#     image_to_movenet(path,keypts_labels_test)    


E:\BE\FinalYearProjectDetails\Yoga_Class-Analysis\dataset\test\Bhujangasana\arthak_bhuj\15.png
[(<tf.Tensor: shape=(1, 1, 17, 3), dtype=float32, numpy=
array([[[[0.7217968 , 0.7860717 , 0.3992842 ],
         [0.7081173 , 0.7939743 , 0.35936993],
         [0.7090212 , 0.79204005, 0.56641805],
         [0.6836773 , 0.77508754, 0.3977951 ],
         [0.68543905, 0.7699059 , 0.50160956],
         [0.6765874 , 0.71619684, 0.5347932 ],
         [0.6961361 , 0.7097354 , 0.48692274],
         [0.72238535, 0.6172997 , 0.42678612],
         [0.75966775, 0.6172461 , 0.3582096 ],
         [0.77743155, 0.63642955, 0.18945757],
         [0.82728094, 0.6293526 , 0.17033398],
         [0.73370576, 0.5085513 , 0.6512314 ],
         [0.7492083 , 0.50123936, 0.47953677],
         [0.74514526, 0.37179875, 0.28392094],
         [0.7695149 , 0.3514926 , 0.3076233 ],
         [0.7351244 , 0.24160558, 0.4099622 ],
         [0.74908257, 0.2241092 , 0.6245212 ]]]], dtype=float32)>, 'Bhujangasana')]
E:\BE\FinalY

KeyboardInterrupt: 